In [1]:
#INSTALL IMPORTANT MODULES
!pip install -q opencv-python
!pip install -q git+https://github.com/tensorflow/docs
!pip install pyngrok
!pip install glob
!pip install jupyter-dash
!sudo -H pip install --upgrade youtube-dl


Error processing line 7 of C:\Users\sachj\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\sachj\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored
Error processing line 7 of C:\Users\sachj\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\sachj\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored


  Stored in directory: C:\Users\sachj\AppData\Local\pip\Cache\wheels\96\02\88\faeae1a755924349309500c9a1d40d5d67df3dc4b4e0d2f5bd
Successfully built pyngrok


Error processing line 7 of C:\Users\sachj\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\sachj\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored


Error processing line 7 of C:\Users\sachj\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\sachj\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored
  ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


Error processing line 7 of C:\Users\sachj\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\sachj\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored
'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
from __future__ import unicode_literals
import os
import glob
import json
import scipy
import tensorflow_hub as hub
from scipy import spatial
from absl import logging
import tensorflow as tf
from tensorflow_docs.vis import embed
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np
from urllib import request  # requires python3
import youtube_dl
import urllib.request
tf.enable_eager_execution()

C:\Users\sachj\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sachj\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sachj\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\sachj\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
#load training data details from JSON (MSR-VTT-10K dataset)
with open('train_val_videodatainfo.json') as f:
          data = json.load(f)

In [3]:
#load pre-computed feature matrices of videos and text from MSR-VTT-10K training dataset
X1 = np.load('feature_matrices/X1.npy')
X2 = np.load('feature_matrices/X2.npy')

#load projection matrices learned from CMFH optimization problem 
P1 = np.load('projection_matrices/P1.npy')
P2 = np.load('projection_matrices/P2.npy')

mu1 = np.mean(X1[:,:],axis=1)
mu2 = np.mean(X2[:,:],axis=1)

mu1 = mu1.reshape(400,1)
mu2 = mu2.reshape(512,1)

#Mean normalization of feature matrices of video(X1) and text (X2)
X1_c = (X1 - mu1) #/std1
X2_c = (X2 - mu2) #/std2

# Compute common embeddings for all videos and text in the training set
r1 = P1.dot(X1_c)
r2 = P2.dot(X2_c)

In [4]:
#print shapes of feature and projection matrices
print(P1.shape)
print(X1.shape)
print(P2.shape)
print(X2.shape)

(48, 400)
(400, 140200)
(48, 512)
(512, 140200)


In [5]:
#load the Universal Sentence Encoder (USE) module
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
use_model = hub.load(module_url)
print ("module %s loaded" % module_url)

def embed1(input):
      #print(type(input))
      return use_model(input)

def semantic_sim(a,b):
    u=embed1(a).numpy()
    v=embed1(b).numpy()
    dist = spatial.distance.cosine(u, v)
    return 1-dist

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [14]:
# get feature matrix of unique videos in the MSR-VTT-10K dataset
r3 = np.array([r1[:,i] for i in range(0,r1.shape[1],20)]).T
print(r3.shape)

# Utilities to open video files using CV2
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = crop_center_square(frame)
      frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)
      
      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0



(48, 7010)


In [15]:
#load i3d model
i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-400/1").signatures['default']

In [16]:
# function to get I3D embeddings for videos
def predict(sample_video):
  # Add a batch axis to the to the sample video.
  model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]
  logits = i3d(model_input)['default'][0]
  probabilities = tf.nn.softmax(logits)
  return np.array([probabilities]).T

In [34]:
def video2text(y_id):
    '''Returns predicted texts on given youtube video

    Args:
        y_id (str): YouTube video ID    
    ''' 
    # New embeddings
    global r1,r2,P1,mu1,data

    video_path = glob.glob("videos/" + y_id + "*")[0]
    video = load_video(video_path)
    feat = predict(video)
    feat_c = (feat - mu1) #/std1
    emb = P1.dot(feat_c) 
    cos_sims = np.array([spatial.distance.cosine(emb,r2[:,j]) for j in range(r2.shape[1])])
    match = np.argsort(cos_sims)[:10]
    pred_texts = [data['sentences'][m]['caption'] for m in match]
    return ([data['sentences'][m]['caption'] for m in match])


[youtube] A30iGQimXnc: Downloading webpage
[youtube] A30iGQimXnc: Downloading MPD manifest
[download] videos\A30iGQimXnc.mp4 has already been downloaded
[download] 100% of 4.65MiB
downloaded!


In [23]:
def text2video(text):
    '''Returns predicted video for given text

    Args:
        text (str): input text    
    ''' 
    # New embeddings
    global data,P1,P2,X1_c,X2_c,mu2,r3

    print(type(embed1([text])))
    feat = embed1([text]).numpy().T
    feat_c = (feat - mu2) #/std1
    emb = P2.dot(feat_c) 
    cos_sims = np.array([spatial.distance.cosine(emb,r3[:,j]) for j in range(r3.shape[1])])
    match = np.argsort(cos_sims)[:10]
    inds = [int(data['sentences'][m*20]['video_id'].replace('video','')) for m in match]
    pred_videos = [data['videos'][ind]['url'] for ind in inds]
    return pred_videos, "Start times in order:{}".format([data['videos'][ind]['start time'] for ind in inds])

In [28]:
# A simple webapp to showcase video-text and text-video retrieval
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State# Load Data
df = px.data.tips()# Build App
#app = JupyterDash(__name__)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H6("Enter YouTube ID to get its predicted text using CMFH!"),

    html.Br(),
    html.Div([
                dcc.Input(id="loading-input-1",placeholder = 'YouTube ID'),
                html.Button(id='submit-button-state', n_clicks=0, children='Submit'),

                dcc.Loading(
                    id="loading-2",
                    children=[html.Div([html.Div(id="loading-output-1")])],
                    type="circle",
                )
                
                
            ]),

    html.Br(),
    html.Br(),
    html.Br(),
    html.Br(),

    html.H6("Enter text to get its predicted video url (YouTube) using CMFH!"),

    html.Br(),
    html.Div([
                dcc.Input(id="loading-input-2",placeholder = 'Text'),
                html.Button(id='submit-button-state1', n_clicks=0, children='Submit'),

                dcc.Loading(
                    id="loading-3",
                    children=[html.Div([html.Div(id="loading-output-2")])],
                    type="circle",
                )
                
                
            ]),

])


@app.callback(Output('loading-output-1', 'children'),
              [Input('submit-button-state', 'n_clicks')],
              [State('loading-input-1', 'value')])

def update_output_div1(n_clicks,input_y_id):
    url = 'https://www.youtube.com/watch?v='+ input_y_id
    ydl_opts = {'outtmpl': 'videos/'+ input_y_id +'.mp4',format: 'mp4'}
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    print("downloaded!")
    return str(video2text(input_y_id))

    
@app.callback(Output('loading-output-2', 'children'),
              [Input('submit-button-state1', 'n_clicks')],
              [State('loading-input-2', 'value')])

def update_output_div2(nclicks, input_txt):
    
      return str(text2video(input_txt))



In [29]:
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
[youtube] A30iGQimXnc: Downloading webpage
[youtube] A30iGQimXnc: Downloading MPD manifest
[download] Destination: videos\A30iGQimXnc.mp4
[download] 100% of 4.65MiB in 00:01                  
downloaded!


In [24]:
text2video("seems there is a road accident")

<class 'tensorflow.python.framework.ops.EagerTensor'>


(['https://www.youtube.com/watch?v=YgmF8as2mFY',
  'https://www.youtube.com/watch?v=-epwvfvWAVI',
  'https://www.youtube.com/watch?v=xnWKz7Cthkk',
  'https://www.youtube.com/watch?v=29ZA7Ag4d_U',
  'https://www.youtube.com/watch?v=K49YyA-LmTc',
  'https://www.youtube.com/watch?v=vE_WqdKbTvY',
  'https://www.youtube.com/watch?v=xOH24qm8JMg',
  'https://www.youtube.com/watch?v=0VTVkaamLmU',
  'https://www.youtube.com/watch?v=BXfJYPsODNU',
  'https://www.youtube.com/watch?v=2aAsvFdf27o'],
 'Start times in order:[25.05, 716.01, 89.85, 366.91, 0.0, 91.45, 28.08, 56.97, 59.88, 4.41]')

In [33]:
video2text('A30iGQimXnc')

IndexError: list index out of range

In [32]:
glob.glob('videos/A30iGQimXnc.mp4')

['videos/A30iGQimXnc.mp4']